In [1]:
import io
import json
import streamlit as st
import pandas as pd
import os
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from googleapiclient.http import MediaFileUpload, MediaIoBaseUpload, MediaIoBaseDownload
from googleapiclient.discovery import build
from google.oauth2 import service_account
import time

from streamlit import columns

# Replace with your file's ID
#list of articles
article_data_id = "1b7eyqlN3lTapBRYcO1VrXGsj_gBVSxQLIyLCPu3UcG8"
#list of districts and alloted funds
district_data_id = "1lwJL-_KQaOY3VSd2cOeOdiR5QOn8yvX3zp6xNfQJo9U"
# past public data
public_data_id = "1sO08BfwN1gzNs_N7XDq1RnqMgJDDKMdq_nsaNhmjKhs"


inst_data_id = "1dOMubywUqJId2gXHwNWp185L3QmadUnwxyFf0DC9M1s"
public_master_id = "1EdEySmYe6ZJUW16f65_q30nkqfbvDADjcmEkAEJrrL4"
master_data_id = "1ry614-7R4-s0uQcv0zrNeS4O0KAbhVEC67rl5_VllGI"

## Access API from local json
creds = service_account.Credentials.from_service_account_file('/Users/aswathshakthi/PycharmProjects/MNP25/data/mnpdatabase-ca1a93fefdd6.json',
        scopes=['https://www.googleapis.com/auth/drive'])

## Access via streamlit secrets
credentials_dict = json.loads(st.secrets["gcp"]["credentials"])
# creds = service_account.Credentials.from_service_account_info(credentials_dict, scopes=['https://www.googleapis.com/auth/drive'])
drive_service = build('drive', 'v3', credentials=creds)


In [2]:
def read_file(file_id):
    try:
        # Specify the desired MIME type for export (e.g., text/csv for Sheets)
        request = drive_service.files().export_media(
            fileId=file_id,
            mimeType='text/csv'  # Change this based on the file type
        )
        file_stream = io.BytesIO()
        downloader = MediaIoBaseDownload(file_stream, request)

        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"Download progress: {int(status.progress() * 100)}%")

        file_stream.seek(0)  # Reset the stream position
        df = pd.read_csv(file_stream)
        return df

    except Exception as e:
        st.error(f"Failed to read file: {e}")

In [3]:
def update_file(file_id, updated_df):
    updated_stream = io.BytesIO()
    updated_df.to_csv(updated_stream, index=False)
    updated_stream.seek(0)

    media = MediaIoBaseUpload(updated_stream, mimetype="text/csv")
    updated_file = drive_service.files().update(
        fileId=file_id,
        media_body=media).execute()

    alert1 = st.success(f"File updated: {updated_file.get('id')}")
    time.sleep(1)
    alert1.empty()

In [4]:
articles_df = read_file(article_data_id)
# Find invalid article names
def valid_articles(one):
    invalid_articles = set(one) - set(articles_df["Articles"])

    # Output result
    if invalid_articles:
        print("These articles are not in the master list:")
        print("\n".join(invalid_articles))
    else:
        print("All articles are valid.")


Download progress: 100%


## District


In [5]:
district_df = read_file(master_data_id)
district_df


Download progress: 100%


,NAME OF THE DISTRICT,REQUESTED ARTICLE,ITEM TYPE,QUANTITY,COST PER UNIT,TOTAL COST,ALLOTTED FUNDS,EXCESS/SHORTAGE,COMMENTS,Beneficiary Type
0,Ariyalur,Agri Battery Sprayer,Article,11,4200.0,46200,190000.0,NaN,No,District
1,Ariyalur,Desktop Computer,Article,1,45500.0,45500,NaN,NaN,No,District
2,Ariyalur,HP Printer 1108 Plus,Article,1,15700.0,15700,NaN,NaN,No,District
3,Ariyalur,Sewing Machine ORD,Article,16,5200.0,83200,NaN,-600.0,No,District
4,Chengalpattu,Agri Power Sprayer (2 STK),Article,2,11760.0,23520,165000.0,NaN,No,District
...,...,...,...,...,...,...,...,...,...,...
396,Virudhunagar,Ex gratia for Deceased,Aid,1,75000.0,75000,NaN,NaN,Rameswari,District
397,Virudhunagar,Medical Aid,Aid,1,50000.0,50000,NaN,NaN,Karuppasamy,District
398,Virudhunagar,Provision materials to Orphanages,Article,1,200000.0,200000,NaN,0.0,Adhiparasakthi elderly aged Home,District
399,Visakapattinam,Bosch Rotary Hammer GBH 220,Article,1,6372.0,6372,20000.0,NaN,No,District


In [6]:
district_df.duplicated(subset=["NAME OF THE DISTRICT","REQUESTED ARTICLE","QUANTITY","COMMENTS"]).any()

np.False_

In [7]:
valid_articles(district_df["REQUESTED ARTICLE"])

All articles are valid.


In [8]:
district_df["NAME OF THE DISTRICT"].nunique()

62

In [9]:
district_df["REQUESTED ARTICLE"].nunique()

84

In [10]:
district_req = district_df.groupby("REQUESTED ARTICLE",as_index=False)["QUANTITY"].sum()

(district_req[district_req["REQUESTED ARTICLE"]=="AHUJA Radios"]["QUANTITY"].to_list()[0])

1

### Public

In [11]:
public_df = read_file(public_master_id).rename(columns={"Article Name": "REQUESTED ARTICLE","Quantity": "QUANTITY"})
public_df

Download progress: 100%


,App. No.,Name,Aadhar (Without Space),Handicapped (Yes / No),Address,Mobile,REQUESTED ARTICLE,Cost Per Unit,QUANTITY,Total Value,Comments,Beneficiary Type,ITEM TYPE
0,P 001,Rekha Elumalai,629795730911,Yes,"96 Palla St, Thiruvannamalai",9003985442,Handicapped Hand Tricycle,7350,1,7350,No,Public,Article
1,P 002,G.J.Arulmanimaran,379283914465,Yes,"5A, First Cross St,Ambattur, Chennai\n",9444278675,Education Aid,15000,0,15000,Rejected,Public,Aid
2,P 003,A.Manikandan,320663697895,No,"390, South St, Kallakurichy",6382220459,Education Aid,15000,1,15000,Rs.15000,Public,Aid
3,P 004,V.Jayavel,548331232628,Yes,"46 Jothi Nagar, Thiruvallur",8668000173,Business Aid,10000,1,10000,Rs.10000,Public,Aid
4,P 005,K.Ganesh,765724849130,No,"Kesavarayanpettai, Melmaruvathur",9655989549,Gents Cycle,5500,1,5500,No,Public,Article
...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,P 252,Agalya,446881190063,No,"Aripakkam, Pondicherry",6374179495,Sewing Machine ORD / Motor,6300,1,6300,No,Public,Article
252,P 253,sakthi Sundaresan,912122436664,No,"Pillaiyarpaliyam, Kanchipuram",6369403054,Education Aid,70000,1,70000,To be paid to APRC Hostel Account,Public,Aid
253,P 254,P.Manikandan,747752435267,Yes,"Naramangani, Villupuram",9884894879,Tea can 10 ltrs,1500,1,1500,No,Public,Article
254,P 255,V.Bhuvanesh,757290226280,No,"Kiliya Nagar, Kanchipuram",6381018032,Laptop,34000,1,34000,No,Public,Article


In [12]:
public_df["REQUESTED ARTICLE"].nunique()

39

In [13]:
valid_articles(public_df["REQUESTED ARTICLE"])

All articles are valid.


In [14]:
public_req = public_df.groupby(["REQUESTED ARTICLE"],as_index=False)["QUANTITY"].sum()
public_req

,REQUESTED ARTICLE,QUANTITY
0,Agri Battery Sprayer,1
1,Agri Cart Weeder,1
2,Aluminium Idli Making Box,1
3,Auto- 3 Wheeler,1
4,Bosch Electrician Kit 13 Re,2
5,Business Aid,33
6,Construction Aid,6
7,Desktop Computer - HVY,1
8,Dictionary English - Tamil,500
9,Education Aid,12


### Institution

In [15]:
inst = read_file(inst_data_id).rename(columns={"Article Name":"REQUESTED ARTICLE","Quantity": "QUANTITY"})
inst_df = inst[inst["Beneficiary Type"]=="Institution"]
inst_df

Download progress: 100%


,App. No.,Institution Name,REQUESTED ARTICLE,QUANTITY,Cost Per Unit,Total Value,Address,Mobile,Comments,Beneficiary Type,ITEM TYPE
16,I 001,Cheyyur Thasildar Office,Laptop - i5,1,60800,60800,"Thasildar Office, Cheyyur, Chengalpattu",9999999999,No,Institution,Article
17,I 002,Adhiparasakthi Institutions,Laptop,81,34000,2754000,Melmaruvathur,044 27529096,No,Institution,Article
18,I 003,Govt School - MKM,Steel Cupboard,2,12390,24780,Marakanam,9444662668,No,Institution,Article
19,I 004,Govt School - KDPM,S Type Chair,5,2242,11210,Kadapakkam,9442826622,No,Institution,Article
20,I 004,Govt School - KDPM,Office Table 4X2,5,4602,23010,Kadapakkam,9442826622,No,Institution,Article
21,I 005,Govt.School - AVPR,Office Table 4X2,5,4602,23010,"Avanipur, Villupuram",8778403593,No,Institution,Article
22,I 006,Govt.School- NMBR,Steel Cupboard,2,12390,24780,Nolambur,94442375680,No,Institution,Article
23,I 007,Govt.School - CYR,Steel Cupboard,3,12390,37170,Cheiyur,8339068395,No,Institution,Article
24,I 008,Anbu Thondu Nilayam,Rice 1000 Kgs,1,60000,60000,Chengalpattu,9865336966,No,Institution,Article


In [16]:
valid_articles(inst_df["REQUESTED ARTICLE"])

All articles are valid.


In [17]:
inst_df["REQUESTED ARTICLE"].nunique()

6

In [18]:
inst_req = inst_df.groupby(["REQUESTED ARTICLE"],as_index=False)["QUANTITY"].sum()
inst_req

,REQUESTED ARTICLE,QUANTITY
0,Laptop,81
1,Laptop - i5,1
2,Office Table 4X2,10
3,Rice 1000 Kgs,1
4,S Type Chair,5
5,Steel Cupboard,7


### Others

In [19]:
oth_df = inst[inst["Beneficiary Type"]=="Others"]
oth_df

,App. No.,Institution Name,REQUESTED ARTICLE,QUANTITY,Cost Per Unit,Total Value,Address,Mobile,Comments,Beneficiary Type,ITEM TYPE
0,A 001,Poongodi,Accident Victim,1,50000,50000,Nil,9629291705,No,Others,Aid
1,A 002,Kannagi Mayazhagu,Accident Victim,1,3000,3000,Kallakurichi,9999999999,A: 339002257674,Others,Aid
2,A 003,Priyanga Gandhi,Accident Victim,1,8000,8000,Kallakurichi,9999999999,A: 545045897495,Others,Aid
3,A 004,Sivagami,Accident Victim,1,10000,10000,Kallakurichi,9999999999,A: 290218141057,Others,Aid
4,A 005,Rani,Accident Victim,1,7500,7500,Kallakurichi,9999999999,A: 907938031037,Others,Aid
5,A 006,Santhi,Accident Victim,1,5000,5000,Kallakurichi,9999999999,A: 906926701129,Others,Aid
6,A 007,Lakshmi,Accident Victim,1,5000,5000,Kallakurichi,9999999999,A: 794881677154,Others,Aid
7,A 008,Tamilarasi,Accident Victim,1,3000,3000,Kallakurichi,9999999999,A: 911476547286,Others,Aid
8,A 009,Ramalingam,Accident Victim,1,8000,8000,Kallakurichi,9999999999,A: 877352990338,Others,Aid
9,A 010,Pazhani,Accident Victim,1,3000,3000,Kallakurichi,9999999999,A: 749232813457,Others,Aid


In [20]:
valid_articles(oth_df["REQUESTED ARTICLE"])

All articles are valid.


In [21]:
oth_df["REQUESTED ARTICLE"].nunique()

2

In [22]:
oth_req = oth_df.groupby(["REQUESTED ARTICLE"],as_index=False)["QUANTITY"].sum()
oth_req

,REQUESTED ARTICLE,QUANTITY
0,Accident Victim,10
1,Flood Relief Fund,6


In [23]:
total_qty = pd.concat([district_req, public_req,inst_req,oth_req]).groupby('REQUESTED ARTICLE', as_index=False).sum()
total_qty

,REQUESTED ARTICLE,QUANTITY
0,3 Ltr Table Top Wet Grinder,1
1,5 HP Bore well Pump / DOL Starter,2
2,AHUJA Radios,1
3,Accident Victim,10
4,Agri Battery Sprayer,119
...,...,...
99,Wet Grinder 3 Ltrs,3
100,Wet Grinder 5 Ltrs,3
101,Wet Grinder Floor Model 2 Ltr,28
102,Wood Saw Machine (P),2


In [24]:
district_df = read_file(master_data_id)[["REQUESTED ARTICLE","QUANTITY","Beneficiary Type"]]
public_df = read_file(public_master_id).rename(columns={"Article Name": "REQUESTED ARTICLE","Quantity": "QUANTITY"})[["REQUESTED ARTICLE","QUANTITY","Beneficiary Type"]]
inst_df = read_file(inst_data_id).rename(columns={"Article Name":"REQUESTED ARTICLE","Quantity": "QUANTITY"})[["REQUESTED ARTICLE","QUANTITY","Beneficiary Type"]]
final = pd.concat([district_df,public_df,inst_df]).reset_index(drop=True).groupby(["REQUESTED ARTICLE","Beneficiary Type"], as_index=False)["QUANTITY"].sum()
final

Download progress: 100%
Download progress: 100%
Download progress: 100%


,REQUESTED ARTICLE,Beneficiary Type,QUANTITY
0,3 Ltr Table Top Wet Grinder,District,1
1,5 HP Bore well Pump / DOL Starter,District,2
2,AHUJA Radios,District,1
3,Accident Victim,Others,10
4,Agri Battery Sprayer,District,118
...,...,...,...
126,Wet Grinder 5 Ltrs,District,3
127,Wet Grinder Floor Model 2 Ltr,District,26
128,Wet Grinder Floor Model 2 Ltr,Public,2
129,Wood Saw Machine (P),District,2


In [25]:
print("District QTY",district_df["QUANTITY"].sum())
print("Public QTY",public_df["QUANTITY"].sum())
print("Inst&Oth QTY",inst_df["QUANTITY"].sum())
print("total qty - merge",final["QUANTITY"].sum())
print("total qty - added",district_df["QUANTITY"].sum() + public_df["QUANTITY"].sum()+inst_df["QUANTITY"].sum())

District QTY 13684
Public QTY 751
Inst&Oth QTY 121
total qty - merge 14556
total qty - added 14556


In [26]:
# Pivot the DataFrame to create columns for each Beneficiary Type and total
pivot_df = final.pivot_table(index="REQUESTED ARTICLE",columns="Beneficiary Type",values="QUANTITY",aggfunc="sum",fill_value=0).reset_index()

# Add missing columns for Beneficiary Types and calculate the Total
pivot_df["District"] = pivot_df.get("District", 0)
pivot_df["Public"] = pivot_df.get("Public", 0)
pivot_df["Institution"] = pivot_df.get("Institution", 0)
pivot_df["Others"] = pivot_df.get("Others", 0)

pivot_df["Total"] = pivot_df[["District", "Public", "Institution", "Others"]].sum(axis=1)
pivot_df
ord_req_id = "1ou21kOkXQpL-hoaJ-11av2m7Kwk5hsif65jVOiFaU2Y"
existing_data = read_file(ord_req_id)

# Ensure required columns exist
if "Ordered Quantity" not in existing_data:
    existing_data["Ordered Quantity"] = 0
if "Remaining Quantity" not in existing_data:
    existing_data["Remaining Quantity"] = existing_data["Total"]

# Merge existing data with the new ordered quantity
updated_df = pivot_df.merge(
            existing_data[["REQUESTED ARTICLE", "Ordered Quantity", "Remaining Quantity"]],
            on="REQUESTED ARTICLE",
            how="left",
            suffixes=("", "_old")
        ).fillna({"Ordered Quantity": 0, "Remaining Quantity": lambda x: x["Total"]})

updated_df

Download progress: 100%


,REQUESTED ARTICLE,District,Institution,Others,Public,Total,Ordered Quantity,Remaining Quantity
0,3 Ltr Table Top Wet Grinder,1,0,0,0,1,0,1
1,5 HP Bore well Pump / DOL Starter,2,0,0,0,2,2,0
2,AHUJA Radios,1,0,0,0,1,1,0
3,Accident Victim,0,0,10,0,10,10,0
4,Agri Battery Sprayer,118,0,0,1,119,125,-6
...,...,...,...,...,...,...,...,...
99,Wet Grinder 3 Ltrs,1,0,0,2,3,3,0
100,Wet Grinder 5 Ltrs,3,0,0,0,3,3,0
101,Wet Grinder Floor Model 2 Ltr,26,0,0,2,28,28,0
102,Wood Saw Machine (P),2,0,0,0,2,2,0


In [27]:
print("District Total",updated_df["District"].sum())
print("Public Total",updated_df["Public"].sum())
print("Institution&oth Total",updated_df["Institution"].sum())


District Total 13684
Public Total 751
Institution&oth Total 105


In [28]:
updated_df["Total"].sum()

np.int64(14556)

### All records Button

In [29]:
district_df = read_file(master_data_id)
district_df["App. No."]=""
cnt = 1
for names in district_df["NAME OF THE DISTRICT"].unique():
    district_df.loc[district_df["NAME OF THE DISTRICT"] == names, "App. No."] = "D " + str(f"{cnt:03}")
    cnt += 1

df1 = district_df[["App. No.","NAME OF THE DISTRICT","REQUESTED ARTICLE","ITEM TYPE","QUANTITY","COST PER UNIT","TOTAL COST","Beneficiary Type","COMMENTS"]]
df1 = df1.rename(columns={"NAME OF THE DISTRICT":"Name","REQUESTED ARTICLE":"Article Name","COST PER UNIT":"Cost Per Unit","QUANTITY":"Quantity","TOTAL COST":"Total Value","COMMENTS":"Comments"})
df1

Download progress: 100%


,App. No.,Name,Article Name,ITEM TYPE,Quantity,Cost Per Unit,Total Value,Beneficiary Type,Comments
0,D 001,Ariyalur,Agri Battery Sprayer,Article,11,4200.0,46200,District,No
1,D 001,Ariyalur,Desktop Computer,Article,1,45500.0,45500,District,No
2,D 001,Ariyalur,HP Printer 1108 Plus,Article,1,15700.0,15700,District,No
3,D 001,Ariyalur,Sewing Machine ORD,Article,16,5200.0,83200,District,No
4,D 002,Chengalpattu,Agri Power Sprayer (2 STK),Article,2,11760.0,23520,District,No
...,...,...,...,...,...,...,...,...,...
396,D 061,Virudhunagar,Ex gratia for Deceased,Aid,1,75000.0,75000,District,Rameswari
397,D 061,Virudhunagar,Medical Aid,Aid,1,50000.0,50000,District,Karuppasamy
398,D 061,Virudhunagar,Provision materials to Orphanages,Article,1,200000.0,200000,District,Adhiparasakthi elderly aged Home
399,D 062,Visakapattinam,Bosch Rotary Hammer GBH 220,Article,1,6372.0,6372,District,No


In [30]:
public_df = read_file(public_master_id)[["App. No.","Name","Article Name","ITEM TYPE","Quantity","Cost Per Unit","Total Value","Beneficiary Type","Comments"]]
public_df

Download progress: 100%


,App. No.,Name,Article Name,ITEM TYPE,Quantity,Cost Per Unit,Total Value,Beneficiary Type,Comments
0,P 001,Rekha Elumalai,Handicapped Hand Tricycle,Article,1,7350,7350,Public,No
1,P 002,G.J.Arulmanimaran,Education Aid,Aid,0,15000,15000,Public,Rejected
2,P 003,A.Manikandan,Education Aid,Aid,1,15000,15000,Public,Rs.15000
3,P 004,V.Jayavel,Business Aid,Aid,1,10000,10000,Public,Rs.10000
4,P 005,K.Ganesh,Gents Cycle,Article,1,5500,5500,Public,No
...,...,...,...,...,...,...,...,...,...
251,P 252,Agalya,Sewing Machine ORD / Motor,Article,1,6300,6300,Public,No
252,P 253,sakthi Sundaresan,Education Aid,Aid,1,70000,70000,Public,To be paid to APRC Hostel Account
253,P 254,P.Manikandan,Tea can 10 ltrs,Article,1,1500,1500,Public,No
254,P 255,V.Bhuvanesh,Laptop,Article,1,34000,34000,Public,No


In [31]:
inst_df = read_file(inst_data_id)[["App. No.","Institution Name","Article Name","ITEM TYPE","Quantity","Cost Per Unit","Total Value","Beneficiary Type","Comments"]]
df3  = inst_df.rename(columns={"Institution Name":"Name"})
df3

Download progress: 100%


,App. No.,Name,Article Name,ITEM TYPE,Quantity,Cost Per Unit,Total Value,Beneficiary Type,Comments
0,A 001,Poongodi,Accident Victim,Aid,1,50000,50000,Others,No
1,A 002,Kannagi Mayazhagu,Accident Victim,Aid,1,3000,3000,Others,A: 339002257674
2,A 003,Priyanga Gandhi,Accident Victim,Aid,1,8000,8000,Others,A: 545045897495
3,A 004,Sivagami,Accident Victim,Aid,1,10000,10000,Others,A: 290218141057
4,A 005,Rani,Accident Victim,Aid,1,7500,7500,Others,A: 907938031037
5,A 006,Santhi,Accident Victim,Aid,1,5000,5000,Others,A: 906926701129
6,A 007,Lakshmi,Accident Victim,Aid,1,5000,5000,Others,A: 794881677154
7,A 008,Tamilarasi,Accident Victim,Aid,1,3000,3000,Others,A: 911476547286
8,A 009,Ramalingam,Accident Victim,Aid,1,8000,8000,Others,A: 877352990338
9,A 010,Pazhani,Accident Victim,Aid,1,3000,3000,Others,A: 749232813457


In [32]:
consold = pd.concat([df1,public_df,df3]).reset_index(drop=True)
consold

,App. No.,Name,Article Name,ITEM TYPE,Quantity,Cost Per Unit,Total Value,Beneficiary Type,Comments
0,D 001,Ariyalur,Agri Battery Sprayer,Article,11,4200.0,46200,District,No
1,D 001,Ariyalur,Desktop Computer,Article,1,45500.0,45500,District,No
2,D 001,Ariyalur,HP Printer 1108 Plus,Article,1,15700.0,15700,District,No
3,D 001,Ariyalur,Sewing Machine ORD,Article,16,5200.0,83200,District,No
4,D 002,Chengalpattu,Agri Power Sprayer (2 STK),Article,2,11760.0,23520,District,No
...,...,...,...,...,...,...,...,...,...
677,I 004,Govt School - KDPM,Office Table 4X2,Article,5,4602.0,23010,Institution,No
678,I 005,Govt.School - AVPR,Office Table 4X2,Article,5,4602.0,23010,Institution,No
679,I 006,Govt.School- NMBR,Steel Cupboard,Article,2,12390.0,24780,Institution,No
680,I 007,Govt.School - CYR,Steel Cupboard,Article,3,12390.0,37170,Institution,No


In [33]:
consold["Quantity"].sum()

np.int64(14556)

In [34]:
district_df



,NAME OF THE DISTRICT,REQUESTED ARTICLE,ITEM TYPE,QUANTITY,COST PER UNIT,TOTAL COST,ALLOTTED FUNDS,EXCESS/SHORTAGE,COMMENTS,Beneficiary Type,App. No.
0,Ariyalur,Agri Battery Sprayer,Article,11,4200.0,46200,190000.0,NaN,No,District,D 001
1,Ariyalur,Desktop Computer,Article,1,45500.0,45500,NaN,NaN,No,District,D 001
2,Ariyalur,HP Printer 1108 Plus,Article,1,15700.0,15700,NaN,NaN,No,District,D 001
3,Ariyalur,Sewing Machine ORD,Article,16,5200.0,83200,NaN,-600.0,No,District,D 001
4,Chengalpattu,Agri Power Sprayer (2 STK),Article,2,11760.0,23520,165000.0,NaN,No,District,D 002
...,...,...,...,...,...,...,...,...,...,...,...
396,Virudhunagar,Ex gratia for Deceased,Aid,1,75000.0,75000,NaN,NaN,Rameswari,District,D 061
397,Virudhunagar,Medical Aid,Aid,1,50000.0,50000,NaN,NaN,Karuppasamy,District,D 061
398,Virudhunagar,Provision materials to Orphanages,Article,1,200000.0,200000,NaN,0.0,Adhiparasakthi elderly aged Home,District,D 061
399,Visakapattinam,Bosch Rotary Hammer GBH 220,Article,1,6372.0,6372,20000.0,NaN,No,District,D 062


In [35]:
district_df[district_df["TOTAL COST"]==0]

,NAME OF THE DISTRICT,REQUESTED ARTICLE,ITEM TYPE,QUANTITY,COST PER UNIT,TOTAL COST,ALLOTTED FUNDS,EXCESS/SHORTAGE,COMMENTS,Beneficiary Type,App. No.
193,Madurai,3 Ltr Table Top Wet Grinder,Article,1,0.0,0,520000.0,NaN,No,District,D 033
195,Madurai,Aluminium Vessels Set - Big,Article,1,0.0,0,NaN,NaN,No,District,D 033
197,Madurai,Ceiling Fan,Article,4,0.0,0,NaN,NaN,No,District,D 033
201,Madurai,Inverter,Article,1,0.0,0,NaN,NaN,No,District,D 033
203,Madurai,Prestige Mixer Grinder 1000 W,Article,2,0.0,0,NaN,NaN,No,District,D 033
204,Madurai,School Furniture,Article,1,0.0,0,NaN,NaN,No,District,D 033


In [36]:
fd = district_df[district_df["ITEM TYPE"]=="Article"]
for i in fd["REQUESTED ARTICLE"].unique():
    if fd[fd["REQUESTED ARTICLE"] == i]["COST PER UNIT"].nunique() == 1:
        print("Good")
    else:
        print(i)


Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Two Wheeler
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Provision materials to Orphanages
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good


In [37]:
district_df = read_file(master_data_id)[["REQUESTED ARTICLE", "QUANTITY", "Beneficiary Type"]]
public_df = read_file(public_master_id).rename(columns={"Article Name": "REQUESTED ARTICLE", "Quantity": "QUANTITY"})[
    ["REQUESTED ARTICLE", "QUANTITY", "Beneficiary Type"]]
inst_df = read_file(inst_data_id).rename(columns={"Article Name": "REQUESTED ARTICLE", "Quantity": "QUANTITY"})[
    ["REQUESTED ARTICLE", "QUANTITY", "Beneficiary Type"]]
final = pd.concat([district_df, public_df, inst_df]).reset_index(drop=True).groupby(
            ["REQUESTED ARTICLE", "Beneficiary Type"], as_index=False).sum()

Download progress: 100%
Download progress: 100%
Download progress: 100%


In [38]:
public_df = read_file(public_master_id)
public_df

Download progress: 100%


,App. No.,Name,Aadhar (Without Space),Handicapped (Yes / No),Address,Mobile,Article Name,Cost Per Unit,Quantity,Total Value,Comments,Beneficiary Type,ITEM TYPE
0,P 001,Rekha Elumalai,629795730911,Yes,"96 Palla St, Thiruvannamalai",9003985442,Handicapped Hand Tricycle,7350,1,7350,No,Public,Article
1,P 002,G.J.Arulmanimaran,379283914465,Yes,"5A, First Cross St,Ambattur, Chennai\n",9444278675,Education Aid,15000,0,15000,Rejected,Public,Aid
2,P 003,A.Manikandan,320663697895,No,"390, South St, Kallakurichy",6382220459,Education Aid,15000,1,15000,Rs.15000,Public,Aid
3,P 004,V.Jayavel,548331232628,Yes,"46 Jothi Nagar, Thiruvallur",8668000173,Business Aid,10000,1,10000,Rs.10000,Public,Aid
4,P 005,K.Ganesh,765724849130,No,"Kesavarayanpettai, Melmaruvathur",9655989549,Gents Cycle,5500,1,5500,No,Public,Article
...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,P 252,Agalya,446881190063,No,"Aripakkam, Pondicherry",6374179495,Sewing Machine ORD / Motor,6300,1,6300,No,Public,Article
252,P 253,sakthi Sundaresan,912122436664,No,"Pillaiyarpaliyam, Kanchipuram",6369403054,Education Aid,70000,1,70000,To be paid to APRC Hostel Account,Public,Aid
253,P 254,P.Manikandan,747752435267,Yes,"Naramangani, Villupuram",9884894879,Tea can 10 ltrs,1500,1,1500,No,Public,Article
254,P 255,V.Bhuvanesh,757290226280,No,"Kiliya Nagar, Kanchipuram",6381018032,Laptop,34000,1,34000,No,Public,Article


In [41]:
public_df.isna().any()

App. No.                  False
Name                      False
Aadhar (Without Space)    False
Handicapped (Yes / No)    False
Address                   False
Mobile                    False
Article Name              False
Cost Per Unit             False
Quantity                  False
Total Value               False
Comments                  False
Beneficiary Type          False
ITEM TYPE                 False
dtype: bool

### Bifurcation of category

In [42]:
ad = articles_df[["Articles","Category"]].rename(columns={"Articles":"Article Name"})
consold = consold.merge(ad, on="Article Name",how='left')
consold

,App. No.,Name,Article Name,ITEM TYPE,Quantity,Cost Per Unit,Total Value,Beneficiary Type,Comments,Category
0,D 001,Ariyalur,Agri Battery Sprayer,Article,11,4200.0,46200,District,No,Agri
1,D 001,Ariyalur,Desktop Computer,Article,1,45500.0,45500,District,No,Computers
2,D 001,Ariyalur,HP Printer 1108 Plus,Article,1,15700.0,15700,District,No,Computers
3,D 001,Ariyalur,Sewing Machine ORD,Article,16,5200.0,83200,District,No,Sewing Machine
4,D 002,Chengalpattu,Agri Power Sprayer (2 STK),Article,2,11760.0,23520,District,No,Agri
...,...,...,...,...,...,...,...,...,...,...
677,I 004,Govt School - KDPM,Office Table 4X2,Article,5,4602.0,23010,Institution,No,Office
678,I 005,Govt.School - AVPR,Office Table 4X2,Article,5,4602.0,23010,Institution,No,Office
679,I 006,Govt.School- NMBR,Steel Cupboard,Article,2,12390.0,24780,Institution,No,Office
680,I 007,Govt.School - CYR,Steel Cupboard,Article,3,12390.0,37170,Institution,No,Office


In [43]:
grouped = consold.groupby(["Category","Article Name"],as_index=False).agg({"Quantity": "sum", "Total Value":"sum"})
# grouped["Category"] = grouped["Category"].mask(grouped["Category"].duplicated(), "")
grouped.to_excel("Full Report.xlsx",index=False)
grouped

,Category,Article Name,Quantity,Total Value
0,1.Project,Project,1,670000
1,2.Aid,Accident Victim,10,102500
2,2.Aid,Business Aid,37,600000
3,2.Aid,Construction Aid,7,320000
4,2.Aid,Education Aid,66,1722350
...,...,...,...,...
99,Sewing Machine,Sewing Machine ORD / Motor,288,1814016
100,Sewing Machine,Sewing Machine Overlock,9,67500
101,Sewing Machine,Sewing Machine Universal ZigZag,2,24600
102,Sewing Machine,Single Burner Stove,4,8400


In [44]:
grouped["Quantity"].sum()

np.int64(14556)

In [45]:
ext =read_file(ord_req_id)
ext

Download progress: 100%


,REQUESTED ARTICLE,District,Institution,Others,Public,Total,Ordered Quantity,Remaining Quantity
0,3 Ltr Table Top Wet Grinder,1,0,0,0,1,0,1
1,5 HP Bore well Pump / DOL Starter,2,0,0,0,2,2,0
2,AHUJA Radios,1,0,0,0,1,1,0
3,Accident Victim,0,0,10,0,10,10,0
4,Agri Battery Sprayer,118,0,0,1,119,125,-6
...,...,...,...,...,...,...,...,...
99,Wet Grinder 3 Ltrs,1,0,0,2,3,3,0
100,Wet Grinder 5 Ltrs,3,0,0,0,3,3,0
101,Wet Grinder Floor Model 2 Ltr,26,0,0,2,28,28,0
102,Wood Saw Machine (P),2,0,0,0,2,2,0


In [64]:
int(ext[ext["REQUESTED ARTICLE"]=="Sewing Machine ORD / Motor"]["District"].values[0])

241

In [62]:
int(ext.loc[ext["REQUESTED ARTICLE"] == "Sewing Machine ORD / Motor", "District"].values[0])


241